Классификация


Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score


Загрузка и первичный анализ данных

In [2]:
df_cancer = pd.read_csv('cancer-risk-factors.csv')
print("Размер датасета:", df_cancer.shape)
print("\nПервые 5 строк:")
print(df_cancer.head())

Размер датасета: (2000, 21)

Первые 5 строк:
  Patient_ID Cancer_Type  Age  Gender  Smoking  Alcohol_Use  Obesity  \
0     LU0000      Breast   68       0        7            2        8   
1     LU0001    Prostate   74       1        8            9        8   
2     LU0002        Skin   55       1        7           10        7   
3     LU0003       Colon   61       0        6            2        2   
4     LU0004        Lung   67       1       10            7        4   

   Family_History  Diet_Red_Meat  Diet_Salted_Processed  ...  \
0               0              5                      3  ...   
1               0              0                      3  ...   
2               0              3                      3  ...   
3               0              6                      2  ...   
4               0              6                      3  ...   

   Physical_Activity  Air_Pollution  Occupational_Hazards  BRCA_Mutation  \
0                  4              6                     3    

Удаляем столбцы

In [3]:
columns_to_drop = ['Overall_Risk_Score', 'Risk_Level', 'Patient_ID']
df_cancer = df_cancer.drop(columns=columns_to_drop)
print("\nСтолбцы после удаления:")
print(df_cancer.columns.tolist())


Столбцы после удаления:
['Cancer_Type', 'Age', 'Gender', 'Smoking', 'Alcohol_Use', 'Obesity', 'Family_History', 'Diet_Red_Meat', 'Diet_Salted_Processed', 'Fruit_Veg_Intake', 'Physical_Activity', 'Air_Pollution', 'Occupational_Hazards', 'BRCA_Mutation', 'H_Pylori_Infection', 'Calcium_Intake', 'BMI', 'Physical_Activity_Level']


Разделяем признаки и целевую переменную

In [4]:
y = df_cancer['Cancer_Type']
X = df_cancer.drop(columns=['Cancer_Type'])

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2000, 17)
y shape: (2000,)


Кодируем целевую переменную

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Классы рака:", label_encoder.classes_)


Классы рака: ['Breast' 'Colon' 'Lung' 'Prostate' 'Skin']


Разделяем данные

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)


Обучаем Decision Tree

In [7]:
dt_baseline = DecisionTreeClassifier(random_state=42)

dt_baseline.fit(X_train, y_train)




,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current

Предсказание

In [8]:
y_pred_baseline = dt_baseline.predict(X_test)


Оценка качества

In [9]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.5875

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.64      0.59      0.61        92
       Colon       0.57      0.55      0.56        84
        Lung       0.69      0.64      0.66       105
    Prostate       0.59      0.59      0.59        61
        Skin       0.42      0.55      0.47        58

    accuracy                           0.59       400
   macro avg       0.58      0.58      0.58       400
weighted avg       0.60      0.59      0.59       400



Улучшение бейзлайна. 

Гипотезы

Ограничение глубины дерева. Без ограничений дерево переобучается.

Минимальное число объектов в листе. Помогает сгладить шум.

Выбор критерия разбиения. gini и entropy могут давать разное качество.

Увелечение min_samples_split сделает дерево стабильнее.

Подбор гиперпараметров (GridSearch)

In [10]:
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_dt = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

grid_dt.fit(X_train, y_train)

print("Лучшие параметры:", grid_dt.best_params_)


Лучшие параметры: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 2}


Модель с улучшенным бейзлайном

In [11]:
best_dt = grid_dt.best_estimator_

y_pred_improved = best_dt.predict(X_test)




Оценка качества

In [12]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))
print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.68

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.69      0.76      0.72        92
       Colon       0.63      0.62      0.62        84
        Lung       0.79      0.75      0.77       105
    Prostate       0.72      0.67      0.69        61
        Skin       0.52      0.52      0.52        58

    accuracy                           0.68       400
   macro avg       0.67      0.66      0.67       400
weighted avg       0.68      0.68      0.68       400



Было

In [13]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.5875

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.64      0.59      0.61        92
       Colon       0.57      0.55      0.56        84
        Lung       0.69      0.64      0.66       105
    Prostate       0.59      0.59      0.59        61
        Skin       0.42      0.55      0.47        58

    accuracy                           0.59       400
   macro avg       0.58      0.58      0.58       400
weighted avg       0.60      0.59      0.59       400



Выводы:

Accuracy, Weighted F1, Macro F1 выросли. 

Гипотезы о необходимости ограничения сложности дерева подтвердились.
Подбор глубины, минимального числа объектов в листьях и критерия разбиения позволил снизить переобучение и улучшить качество классификации по всем метрикам.

Имплементация алгоритма машинного обучения


In [14]:
def gini(y):
    counts = np.bincount(y)
    probs = counts / len(y)
    return 1 - np.sum(probs ** 2)

class MyDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _best_split(self, X, y):
        best_gini = float('inf')
        best_feature, best_threshold = None, None

        n_samples, n_features = X.shape

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left = y[X[:, feature] <= t]
                right = y[X[:, feature] > t]

                if len(left) == 0 or len(right) == 0:
                    continue

                g = (len(left) / n_samples) * gini(left) + \
                    (len(right) / n_samples) * gini(right)

                if g < best_gini:
                    best_gini = g
                    best_feature = feature
                    best_threshold = t

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        num_samples = len(y)
        num_classes = len(np.unique(y))

        if (
            num_classes == 1 or
            num_samples < self.min_samples_split or
            (self.max_depth is not None and depth >= self.max_depth)
        ):
            return Counter(y).most_common(1)[0][0]

        feature, threshold = self._best_split(X, y)

        if feature is None:
            return Counter(y).most_common(1)[0][0]

        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def _predict_one(self, x, node):
        if not isinstance(node, dict):
            return node

        if x[node['feature']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])


Обучение и подготовка данных

In [17]:
X_np = X.values
y_np = y_encoded

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np,
    y_np,
    test_size=0.2,
    random_state=42,
    stratify=y_np
)

my_dt_baseline = MyDecisionTreeClassifier()

my_dt_baseline.fit(X_train_np, y_train_np)

y_pred_my_base = my_dt_baseline.predict(X_test_np)


Оценка качества

In [19]:
print("My DT Baseline Accuracy:", accuracy_score(y_test_np, y_pred_my_base))
print("\nMy DT Baseline Classification Report:")
print(classification_report(
    y_test_np,
    y_pred_my_base,
    target_names=label_encoder.classes_
))

My DT Baseline Accuracy: 0.595

My DT Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.67      0.62      0.64        92
       Colon       0.59      0.57      0.58        84
        Lung       0.68      0.62      0.65       105
    Prostate       0.56      0.57      0.57        61
        Skin       0.43      0.57      0.49        58

    accuracy                           0.59       400
   macro avg       0.59      0.59      0.59       400
weighted avg       0.61      0.59      0.60       400



Было у алгоритма из sklearn


In [20]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.5875

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.64      0.59      0.61        92
       Colon       0.57      0.55      0.56        84
        Lung       0.69      0.64      0.66       105
    Prostate       0.59      0.59      0.59        61
        Skin       0.42      0.55      0.47        58

    accuracy                           0.59       400
   macro avg       0.58      0.58      0.58       400
weighted avg       0.60      0.59      0.59       400



Выводы: собственный алгоритм показал небольшое улучшение относительно бейзлана из sklearn, что подтверждает корректность реализации.

Улучшение MyKNN

Ограничение глубины уменьшит переобучение

Увеличение min_samples_split сделает дерево стабильнее

In [21]:
my_dt_improved = MyDecisionTreeClassifier(
    max_depth=7,
    min_samples_split=10
)

my_dt_improved.fit(X_train_np, y_train_np)

y_pred_my_imp = my_dt_improved.predict(X_test_np)


Оценка


In [22]:
print("My DT Improved Accuracy:", accuracy_score(y_test_np, y_pred_my_imp))
print("\nMy DT Improved Classification Report:")
print(classification_report(
    y_test_np,
    y_pred_my_imp,
    target_names=label_encoder.classes_
))

My DT Improved Accuracy: 0.685

My DT Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.71      0.75      0.73        92
       Colon       0.65      0.62      0.63        84
        Lung       0.79      0.75      0.77       105
    Prostate       0.72      0.64      0.68        61
        Skin       0.51      0.60      0.55        58

    accuracy                           0.69       400
   macro avg       0.68      0.67      0.67       400
weighted avg       0.69      0.69      0.69       400



Было у улучшенного бейзлайна из sklearn

In [23]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))
print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.68

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.69      0.76      0.72        92
       Colon       0.63      0.62      0.62        84
        Lung       0.79      0.75      0.77       105
    Prostate       0.72      0.67      0.69        61
        Skin       0.52      0.52      0.52        58

    accuracy                           0.68       400
   macro avg       0.67      0.66      0.67       400
weighted avg       0.68      0.68      0.68       400



Вывод: Результаты у собственной реализации незначительно лучше чем  у алгоритма из sklearn  после применения гипотез, что подтверждает корректность написанного алгоритма.

Регрессия


Загрузка и первичный анализ данных

In [26]:
df_productivity = pd.read_csv('social_media_vs_productivity.csv')
print("Размер датасета:", df_productivity.shape)
print("\nПервые 5 строк:")
print(df_productivity.head())

Размер датасета: (30000, 19)

Первые 5 строк:
   age  gender    job_type  daily_social_media_time  \
0   56    Male  Unemployed                 4.180940   
1   46    Male      Health                 3.249603   
2   32    Male     Finance                      NaN   
3   60  Female  Unemployed                      NaN   
4   25    Male          IT                      NaN   

  social_platform_preference  number_of_notifications  work_hours_per_day  \
0                   Facebook                       61            6.753558   
1                    Twitter                       59            9.169296   
2                    Twitter                       57            7.910952   
3                   Facebook                       59            6.355027   
4                   Telegram                       66            6.214096   

   perceived_productivity_score  actual_productivity_score  stress_level  \
0                      8.040464                   7.291555           4.0   
1       

Обработка пропусков и уменьшения размера датасета


In [27]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

df_productivity_clean = df_productivity.dropna(
    subset=['actual_productivity_score']
)

df_productivity_clean = df_productivity_clean.iloc[:1000].copy()




Разделяем признаки и целевую переменную

In [28]:
target = 'actual_productivity_score'

X = df_productivity_clean.drop(columns=[target])
y = df_productivity_clean[target]


Препроцессинг



In [29]:
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)


Train / Test split

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Baseline Decision Tree Regressor

In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline


dt_reg_baseline = Pipeline([
    ('preprocessor', preprocessor),
    ('dt', DecisionTreeRegressor(random_state=42))
])

dt_reg_baseline.fit(X_train, y_train)

y_pred_baseline = dt_reg_baseline.predict(X_test)


Оценка качества

In [32]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))


Baseline MSE: 0.5353353179748289
Baseline R2: 0.8415468482971886


Улучшение бейзлайна

Гипотезы

Гипотеза 1
Ограничение глубины дерева снизит переобучение

Гипотеза 2
Увеличение min_samples_leaf сделает предсказания более стабильными

Гипотеза 3
Подбор max_depth и min_samples_split улучшит MSE

In [33]:
pipe_dt = Pipeline([
    ('preprocessor', preprocessor),
    ('dt', DecisionTreeRegressor(random_state=42))
])


Подбор гиперпараметров

In [34]:
param_grid = {
    'dt__max_depth': [3, 5, 7, 10, None],
    'dt__min_samples_split': [2, 5, 10, 20],
    'dt__min_samples_leaf': [1, 5, 10]
}

grid_dt = GridSearchCV(
    pipe_dt,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_dt.fit(X_train, y_train)

print("Лучшие параметры:", grid_dt.best_params_)


Лучшие параметры: {'dt__max_depth': 5, 'dt__min_samples_leaf': 10, 'dt__min_samples_split': 2}


Оценка улучшенного бейзлайна

In [35]:
best_dt_reg = grid_dt.best_estimator_

y_pred_improved = best_dt_reg.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))


Improved MSE: 0.3209938715659406
Improved R2: 0.904989473103842


Было

In [36]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))


Baseline MSE: 0.5353353179748289
Baseline R2: 0.8415468482971886


Выводы: улучшенный бейзлайн Decision Tree показал снижение MSE и рост R².Это подтверждает гипотезу о том, что ограничение сложности дерева уменьшает переобучение и улучшает обобщающую способность модели.

Собственная реализация Decision Tree Regressor

In [38]:
class MyDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y):
        best_mse = float('inf')
        best_feature, best_threshold = None, None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left = y[X[:, feature] <= t]
                right = y[X[:, feature] > t]

                if len(left) == 0 or len(right) == 0:
                    continue

                mse = (len(left) * self._mse(left) +
                       len(right) * self._mse(right)) / len(y)

                if mse < best_mse:
                    best_mse = mse
                    best_feature = feature
                    best_threshold = t

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        if (
            len(y) < self.min_samples_split or
            (self.max_depth is not None and depth >= self.max_depth)
        ):
            return np.mean(y)

        feature, threshold = self._best_split(X, y)

        if feature is None:
            return np.mean(y)

        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def _predict_one(self, x, node):
        if not isinstance(node, dict):
            return node

        if x[node['feature']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])


Подготовка данных  (используем только числовые признаки)

In [39]:
X_num = df_productivity_clean[numeric_features]
y_num = df_productivity_clean[target]

X_num = SimpleImputer(strategy='mean').fit_transform(X_num)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_num, y_num, test_size=0.2, random_state=42
)


Обучение и оценка

In [40]:
my_dt_reg_base = MyDecisionTreeRegressor()

my_dt_reg_base.fit(X_train_n, y_train_n.values)

y_pred_my_base = my_dt_reg_base.predict(X_test_n)

print("My DT Baseline MSE:", mean_squared_error(y_test_n, y_pred_my_base))
print("My DT Baseline R2:", r2_score(y_test_n, y_pred_my_base))


My DT Baseline MSE: 0.5466626356213844
My DT Baseline R2: 0.8381940913966622


Было у алгоритма из sklearn

In [41]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))


Baseline MSE: 0.5353353179748289
Baseline R2: 0.8415468482971886


Вывод: собственная реализация оказалась немного хуже. Это обьясняется наивной реализацией(только числовые признаки, редсказание - среднее в листе)


Используем гипотезы для собственного алгоритма(ограничение глубины уменьшит переобучение, увеличение min_samples_split стабилизирует предсказания)

In [42]:
my_dt_reg_improved = MyDecisionTreeRegressor(
    max_depth=6,
    min_samples_split=20
)

my_dt_reg_improved.fit(X_train_n, y_train_n.values)

y_pred_my_imp = my_dt_reg_improved.predict(X_test_n)

print("My DT Improved MSE:", mean_squared_error(y_test_n, y_pred_my_imp))
print("My DT Improved R2:", r2_score(y_test_n, y_pred_my_imp))


My DT Improved MSE: 0.3347494828512464
My DT Improved R2: 0.9009179689669565


Было у улучшенного бейзлайна из sklearn

In [43]:
best_dt_reg = grid_dt.best_estimator_

y_pred_improved = best_dt_reg.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))


Improved MSE: 0.3209938715659406
Improved R2: 0.904989473103842


Вывод: Результаты у улучшенного собственного алгоритма оказались немного хуже улучшенного алгоритма из sklearn, что обьясняется упращенности реализации(только числовые признаки, редсказание - среднее в листе).